# Loosely-coupled loop closure

Attempted to use GTSAM EssentialMatrixFactor with side camera pairs, but the EssentialMatrixFactor
uses and essential matrix key and it is unclear how it interacts with pose keys.


In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import cv2
from gtsam.symbol_shorthand import X
import plotly.graph_objects as go

from lightglue import viz2d

from lac.slam.feature_tracker import FeatureTracker
from lac.slam.slam import PoseGraph
from lac.slam.visual_odometry import StereoVisualOdometry
from lac.utils.plotting import plot_poses, plot_lander_3d
from lac.utils.visualization import image_grid
from lac.util import load_data

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = "../../../output/DataCollectionAgent/double_loop_preset1"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

fig = plot_poses(poses, no_axes=True, color="black", name="Ground truth")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

# Pose graph

- Run VO and add odometry factors to graph. Designate every N-th pose as a keyframe (for loop closure checking)
- For each new pose, check its distance to all other keyframes excluding most recent ones. If that distance is less
  than a threshold, check the angle between the two poses. If that angle is less than a threshold, attempt to estimate
  a relative pose for loop closure. If LightGlue finds sufficient matches and PnP is successful, add a loop closure factor.
- If the loop closure factor is added, run optimization on the graph.


In [ ]:
graph = PoseGraph()
graph.add_pose(0, initial_pose)

# Run VO to get odometry


In [ ]:
# Load the data logs
data_path = "../../../output/DataCollectionAgent/double_loop_preset1"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
left_path = Path(data_path) / "FrontLeft"
right_path = Path(data_path) / "FrontRight"
side_path = Path(data_path) / "Right"

In [ ]:
svo = StereoVisualOdometry(cam_config)
svo_poses = []
pose_deltas = []

START_FRAME = 80
END_FRAME = len(poses)

print("Running VO...")
progress_bar = tqdm(range(START_FRAME, END_FRAME, 2), dynamic_ncols=True)

for frame in progress_bar:
    progress_bar.set_description(f"Processing Frame: {frame}")

    img_name = f"{frame:06}.png"
    left_img = cv2.imread(str(left_path / img_name), cv2.IMREAD_GRAYSCALE)
    right_img = cv2.imread(str(right_path / img_name), cv2.IMREAD_GRAYSCALE)

    if frame == START_FRAME:
        svo.initialize(poses[frame], left_img, right_img)
        svo_poses.append(poses[frame])
        continue

    svo.track(left_img, right_img)
    svo_poses.append(svo.rover_pose)
    pose_deltas.append(svo.pose_delta)

In [ ]:
frame_to_index = {frame: i for i, frame in enumerate(range(START_FRAME, END_FRAME, 2))}
index_to_frame = {i: frame for i, frame in enumerate(range(START_FRAME, END_FRAME, 2))}

In [ ]:
fig = go.Figure()
fig = plot_poses(poses[START_FRAME:END_FRAME], fig=fig, no_axes=True, color="black", name="Ground truth")
fig = plot_poses(svo_poses, fig=fig, no_axes=True, color="orange", name="VO")
fig = plot_lander_3d(fig=fig, lander_height=lander_pose[2, 3], color="silver")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

# Detect loop closures

Position based


In [ ]:
tracker = FeatureTracker(cam_config)

In [ ]:
i1 = 987
i2 = 2297
frame1 = index_to_frame[i1]
frame2 = index_to_frame[i2]

side_img1 = cv2.imread(str(side_path / f"{frame1:06}.png"), cv2.IMREAD_GRAYSCALE)
side_img2 = cv2.imread(str(side_path / f"{frame2:06}.png"), cv2.IMREAD_GRAYSCALE)

# Feature matching
feats1 = tracker.extract_feats(side_img1)
feats2 = tracker.extract_feats(side_img2)
matches = tracker.match_feats(feats1, feats2)

points1 = feats1["keypoints"][0][matches[:, 0]].cpu().numpy()
points2 = feats2["keypoints"][0][matches[:, 1]].cpu().numpy()

viz2d.plot_images([side_img1, side_img2])
viz2d.plot_matches(points1, points2, lw=0.2)

In [ ]:
indices = [284, 300, 615]
indices += [1604, 1580, 2148]
images = []

for i in indices:
    images.append(cv2.imread(str(side_path / f"{index_to_frame[i]:06}.png"), cv2.IMREAD_GRAYSCALE))

image_grid(images, rows=2, cols=len(images) // 2)

In [ ]:
# TODO: get a list of LC pairs
loop_closures = [
    (284, 1604),
    (300, 1580),
    (615, 2148),
    (1025, 2648),
    (1319, 3200),
    (1319, 3300),
    (1565, 3643),
]

In [ ]:
fig = plot_poses(poses[START_FRAME:END_FRAME], no_axes=True, color="black", name="Ground truth")
fig = plot_poses(svo_poses, fig=fig, no_axes=True, color="orange", name="VO")
for i, j in loop_closures:
    fig.add_trace(
        go.Scatter3d(
            x=[svo_poses[i][0, 3], svo_poses[j][0, 3]],
            y=[svo_poses[i][1, 3], svo_poses[j][1, 3]],
            z=[svo_poses[i][2, 3], svo_poses[j][2, 3]],
            mode="markers+lines",
            marker=dict(color="red", size=5),
            line=dict(color="red", width=5),
            name=f"LC {i}-{j}",
        )
    )
fig = plot_lander_3d(fig=fig, lander_height=lander_pose[2, 3], color="silver")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

In [ ]:
fig.write_html("../../../results/slam/manual_loop_closures.html")

# Add loop closures

With gtsam EssentialMatrixFactor


In [ ]:
import gtsam
from gtsam.symbol_shorthand import X

from lac.slam.loop_closure import estimate_loop_closure_pose
from lac.slam.slam import K

In [ ]:
graph = gtsam.NonlinearFactorGraph()
values = gtsam.Values()

sigma_t = 0.005  # [m]
sigma_R = 0.00087  # [rad]
svo_pose_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([sigma_R, sigma_R, sigma_R, sigma_t, sigma_t, sigma_t]))

values.insert(X(0), gtsam.Pose3(initial_pose))
graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(initial_pose)))

i = 1
for frame in tqdm(np.arange(START_FRAME + 2, END_FRAME, 2)):
    values.insert(X(i), gtsam.Pose3(svo_poses[i]))
    graph.push_back(gtsam.BetweenFactorPose3(X(i - 1), X(i), gtsam.Pose3(pose_deltas[i - 1]), svo_pose_noise))
    i += 1

Essential matrix factor


In [ ]:
# noise_model = gtsam.noiseModel.Isotropic.Sigma(2, 3.0)

# for i, j in loop_closures:
#     img_i = cv2.imread(str(side_path / f"{index_to_frame[i]:06}.png"), cv2.IMREAD_GRAYSCALE)
#     img_j = cv2.imread(str(side_path / f"{index_to_frame[j]:06}.png"), cv2.IMREAD_GRAYSCALE)
#     feats_i = tracker.extract_feats(img_i)
#     feats_j = tracker.extract_feats(img_j)
#     matches = tracker.match_feats(feats_i, feats_j)
#     matched_kps_i = feats_i["keypoints"][0][matches[:, 0]].cpu().numpy()
#     matched_kps_j = feats_j["keypoints"][0][matches[:, 1]].cpu().numpy()

#     graph.add(gtsam.EssentialMatrixFactor(X(i), X(j), matched_kps_i, matched_kps_j, K))

PnP relative pose estimation


In [ ]:
noise_model = gtsam.noiseModel.Isotropic.Sigma(2, 3.0)

for i, j in loop_closures:
    img_i = cv2.imread(str(side_path / f"{index_to_frame[i]:06}.png"), cv2.IMREAD_GRAYSCALE)
    img_j = cv2.imread(str(side_path / f"{index_to_frame[j]:06}.png"), cv2.IMREAD_GRAYSCALE)
    feats_i = tracker.extract_feats(img_i)
    feats_j = tracker.extract_feats(img_j)
    matches = tracker.match_feats(feats_i, feats_j)
    matched_kps_i = feats_i["keypoints"][0][matches[:, 0]].cpu().numpy()
    matched_kps_j = feats_j["keypoints"][0][matches[:, 1]].cpu().numpy()

    graph.add(gtsam.EssentialMatrixFactor(X(i), X(j), matched_kps_i, matched_kps_j, K))